In [0]:
# Camada Bronze - SubTypes - Magic: The Gathering 
# Objetivo: Processo EL (Extract & Load) com nomenclatura padronizada
# Características: Extract da staging (S3/Parquet) -> Load na bronze (Unity Catalog/Delta) 
# Melhorias: Nomenclatura padronizada, colunas padronizadas, merge incremental robusto

# =============================================================================
# BIBLIOTECAS UTILIZADAS
# =============================================================================
import logging
from datetime import datetime, timedelta
from pyspark.sql.functions import *
from delta.tables import DeltaTable

# =============================================================================
# CONFIGURAÇÃO DE SEGREDOS
# =============================================================================
def get_secret(secret_name, default_value=None):
    try:
        return dbutils.secrets.get(scope="mtg-pipeline", key=secret_name)
    except:
        if default_value is not None:
            print(f"Segredo '{secret_name}' não encontrado, usando valor padrão")
            return default_value
        else:
            print(f"Segredo obrigatório '{secret_name}' não configurado")
            raise Exception(f"Segredo '{secret_name}' não configurado")

# =============================================================================
# CONFIGURAÇÕES GLOBAIS
# =============================================================================
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

CATALOG_NAME = get_secret("catalog_name")
SCHEMA_NAME = "bronze"
TABLE_NAME = "TB_BRONZE_SUBTYPES"

S3_BUCKET = get_secret("s3_bucket")
S3_STAGE_PREFIX = get_secret("s3_stage_prefix", "magic_the_gathering/stage")
S3_BRONZE_PREFIX = get_secret("s3_bronze_prefix", "magic_the_gathering/bronze")
S3_STAGE_PATH = f"s3://{S3_BUCKET}/{S3_STAGE_PREFIX}"
S3_BRONZE_PATH = f"{S3_BUCKET}/{S3_BRONZE_PREFIX}"

print("Iniciando pipeline EL Unity com nomenclatura padronizada - TB_BRONZE_SUBTYPES")
print("Dados de referência MTG API - sem filtro temporal")

# =============================================================================
# MAPEAMENTO DE COLUNAS GOV
# =============================================================================
# Mapeamento das colunas originais para nomenclatura GOV (alinhado com Silver)
COLUMN_MAPPING = {
    # Identificação
    'subtype_name': 'NME_SUBTYPE',
    
    # Metadados
    'source': 'NME_SOURCE',
    'ingestion_timestamp': 'DT_INGESTION',
    
    # Datas
    'created_at': 'DT_CREATED',
    'updated_at': 'DT_UPDATED'
}

# =============================================================================
# FUNÇÕES UTILITÁRIAS
# =============================================================================
def setup_unity_catalog():
    # Configura o Unity Catalog e schema
    try:
        spark.sql(f"CREATE CATALOG IF NOT EXISTS {CATALOG_NAME}")
        spark.sql(f"USE CATALOG {CATALOG_NAME}")
        spark.sql(f"CREATE SCHEMA IF NOT EXISTS {SCHEMA_NAME}")
        spark.sql(f"USE SCHEMA {SCHEMA_NAME}")
        return True
    except Exception as e:
        print(f"Erro ao configurar Unity Catalog: {e}")
        return False

def extract_from_staging(table_name):
    # EXTRACT: Lê dados da camada staging
    try:
        print(f"Iniciando extract_from_staging para {table_name}")
        
        # Para TB_BRONZE_SUBTYPES, ler do arquivo subtypes da staging
        if table_name == "TB_BRONZE_SUBTYPES":
            stage_path = f"{S3_STAGE_PATH}/*_subtypes.parquet"
        else:
            stage_path = f"{S3_STAGE_PATH}/*_{table_name}.parquet"
        
        print(f"Tentando ler dados de: {stage_path}")
        
        df = spark.read.parquet(stage_path)
        count = df.count()
        print(f"Extraídos {count} registros de staging para {table_name}")
        
        if count == 0:
            print("AVISO: Nenhum registro encontrado em staging")
            return None
            
        return df
    except Exception as e:
        print(f"Erro no EXTRACT de staging: {e}")
        print(f"Path tentado: {stage_path}")
        return None

def apply_governance_naming(df):
    # Aplica nomenclatura GOV nas colunas
    try:
        # Obter colunas disponíveis no DataFrame
        available_columns = df.columns
        
        # Criar mapeamento apenas para colunas que existem
        column_mapping = {}
        for original_col, gov_col in COLUMN_MAPPING.items():
            if original_col in available_columns:
                column_mapping[original_col] = gov_col
        
        # Renomear colunas
        for original_col, gov_col in column_mapping.items():
            df = df.withColumnRenamed(original_col, gov_col)
        
        print(f"Aplicada nomenclatura GOV em {len(column_mapping)} colunas")
        print(f"Colunas renomeadas: {list(column_mapping.keys())}")
        
        return df
    except Exception as e:
        print(f"Erro ao aplicar nomenclatura GOV: {e}")
        return df

def apply_temporal_filter(df, table_name):
    # Para dados de referência MTG API, não aplicar filtro temporal
    if table_name == "TB_BRONZE_SUBTYPES":
        print("Dados de referência MTG API - filtro temporal não aplicado")
        return df
    return df

def check_delta_table_exists(delta_path):
    # Verifica se a tabela Delta existe
    try:
        # Verificar se _delta_log existe
        delta_log_path = f"{delta_path}/_delta_log"
        try:
            files = dbutils.fs.ls(delta_log_path)
            if files:
                # Tentar ler a tabela para verificar se tem dados
                df = spark.read.format("delta").load(delta_path)
                count = df.count()
                print(f"Tabela Delta existe com {count} registros")
                return True, count > 0
            else:
                return False, False
        except:
            return False, False
    except Exception as e:
        print(f"Erro ao verificar tabela Delta: {e}")
        return False, False

def get_delta_schema_for_merge(delta_path):
    # Obtém o schema da tabela Delta existente, excluindo colunas de particionamento
    try:
        df = spark.read.format("delta").load(delta_path)
        schema_fields = [field.name for field in df.schema.fields]
        
        # Excluir colunas de particionamento do schema para merge
        partition_columns = ['INGESTION_YEAR', 'INGESTION_MONTH']
        merge_schema = [col for col in schema_fields if col not in partition_columns]
        
        print(f"Schema completo da tabela Delta: {schema_fields}")
        print(f"Schema para merge (sem particionamento): {merge_schema}")
        return merge_schema
    except Exception as e:
        print(f"Erro ao obter schema Delta: {e}")
        return []

def prepare_dataframe_for_merge(df, table_name, target_schema=None):
    # Prepara DataFrame para merge, garantindo compatibilidade de schema
    if not df:
        return None
    try:
        # Aplica nomenclatura GOV
        df = apply_governance_naming(df)

        # Remove coluna 'endpoint' se existir
        if 'endpoint' in df.columns:
            df = df.drop('endpoint')
            print("Coluna 'endpoint' removida do DataFrame")

        # Elimina duplicidade de NME_SUBTYPE no DataFrame de origem
        total_before = df.count()
        df = df.dropDuplicates(['NME_SUBTYPE'])
        total_after = df.count()
        print(f"Removidas {total_before - total_after} duplicatas baseadas em 'NME_SUBTYPE'")

        # Aplicar filtro temporal (não aplicado para dados de referência)
        df = apply_temporal_filter(df, table_name)

        # Se temos schema de destino, filtrar colunas compatíveis
        if target_schema:
            available_columns = df.columns
            compatible_columns = [col for col in available_columns if col in target_schema]

            if not compatible_columns:
                print("Nenhuma coluna compatível encontrada para merge")
                return None

            print(f"Colunas compatíveis para merge: {compatible_columns}")
            df = df.select(compatible_columns)

        return df
    except Exception as e:
        print(f"Erro ao preparar DataFrame para merge: {e}")
        return None

def execute_merge_with_specific_columns(delta_table, merge_df, merge_columns):
    # Executa merge especificando exatamente quais colunas atualizar
    try:
        # Construir a condição de merge (subtypes usa 'NME_SUBTYPE' como chave)
        merge_condition = "bronze.NME_SUBTYPE = novo.NME_SUBTYPE"
        
        # Construir as ações de UPDATE especificando colunas
        update_actions = {}
        for col_name in merge_columns:
            if col_name != 'NME_SUBTYPE':  # Não atualizar a chave de merge
                update_actions[col_name] = f"novo.{col_name}"
        
        # Construir as ações de INSERT especificando colunas
        insert_actions = {}
        for col_name in merge_columns:
            insert_actions[col_name] = f"novo.{col_name}"
        
        print(f"Executando merge com {len(update_actions)} colunas para UPDATE")
        print(f"Colunas para UPDATE: {list(update_actions.keys())}")
        
        # Executar merge com ações específicas
        delta_table.alias("bronze").merge(
            merge_df.alias("novo"),
            merge_condition
        ).whenMatchedUpdate(set=update_actions) \
         .whenNotMatchedInsert(values=insert_actions) \
         .execute()
        
        print("Merge Delta executado com sucesso")
        return True
    except Exception as e:
        print(f"Erro no merge Delta: {e}")
        return False

def check_unity_table_exists(full_table_name):
    # Verifica se a tabela Unity Catalog existe
    try:
        # Tentar fazer uma consulta simples na tabela
        test_query = f"SELECT 1 FROM {full_table_name} LIMIT 1"
        spark.sql(test_query)
        print(f"Tabela Unity Catalog '{full_table_name}' existe")
        return True
    except Exception as e:
        print(f"Tabela Unity Catalog '{full_table_name}' não existe ou não está acessível")
        return False

def create_or_update_unity_catalog(full_table_name, delta_path):
    # Cria ou atualiza tabela Unity Catalog preservando modificações existentes
    try:
        table_exists = check_unity_table_exists(full_table_name)
        
        if not table_exists:
            # Criar tabela pela primeira vez
            print(f"Criando tabela Unity Catalog: {full_table_name}")
            create_table_sql = f"""
            CREATE TABLE IF NOT EXISTS {full_table_name}
            USING DELTA
            LOCATION '{delta_path}'
            COMMENT 'Tabela bronze de referência de subtypes do Magic: The Gathering com nomenclatura padronizada'
            """
            spark.sql(create_table_sql)
            print(f"Tabela Unity Catalog criada: {full_table_name}")
        else:
            # Tabela já existe, apenas atualizar propriedades de pipeline (sem perder modificações)
            print(f"Tabela Unity Catalog já existe: {full_table_name}")
            print("Atualizando apenas propriedades de pipeline (preservando modificações existentes)")
        
        # Sempre atualizar propriedades de pipeline (não afeta modificações customizadas)
        try:
            spark.sql(f"""
            ALTER TABLE {full_table_name} SET TBLPROPERTIES (
                'bronze_layer' = 'true',
                'data_source' = 'mtg_reference',
                'last_processing_date' = '{datetime.now().strftime("%Y-%m-%d %H:%M:%S")}',
                'table_type' = 'bronze',
                'load_mode' = 'incremental_merge_gov',
                'governance_applied' = 'true',
                'naming_convention' = 'GOV',
                'temporal_filter' = 'none',
                'partitioning' = 'INGESTION_YEAR_MONTH'
            )
            """)
            print("Propriedades de pipeline atualizadas")
        except Exception as e:
            print(f"Erro ao atualizar propriedades de pipeline: {e}")
        
        return True
    except Exception as e:
        print(f"Erro ao criar/atualizar Unity Catalog: {e}")
        return False

def load_to_bronze_unity_incremental_gov(df, table_name):
    # LOAD: Carrega dados na camada bronze (Unity + Incremental + GOV)
    if not df:
        return None
    try:
        delta_path = f"s3://{S3_BRONZE_PATH}/{table_name}"
        full_table_name = f"{CATALOG_NAME}.{SCHEMA_NAME}.{table_name}"

        from delta.tables import DeltaTable

        print(f"Carregando dados para bronze: {table_name}")
        print(f"Unity Catalog: {full_table_name}")
        print(f"Delta Path: {delta_path}")

        # Verificar se a tabela Delta existe
        delta_exists, has_data = check_delta_table_exists(delta_path)

        if not delta_exists or not has_data:
            # Primeira criação com particionamento
            print("Criando tabela Delta pela primeira vez com nomenclatura GOV...")
            df_processed = prepare_dataframe_for_merge(df, table_name)

            from pyspark.sql.functions import year, month, current_timestamp

            df_with_partition = df_processed.withColumn("DT_INGESTION", current_timestamp()) \
                                           .withColumn("INGESTION_YEAR", year(col("DT_INGESTION"))) \
                                           .withColumn("INGESTION_MONTH", month(col("DT_INGESTION")))

            print(f"Particionamento criado com DT_INGESTION")

            df_with_partition.write.format("delta") \
                   .mode("overwrite") \
                   .option("overwriteSchema", "true") \
                   .partitionBy("INGESTION_YEAR", "INGESTION_MONTH") \
                   .save(delta_path)
            print("Tabela Delta criada com sucesso")
        else:
            # Verificar se o schema está correto
            print("Tabela Delta existe, verificando schema...")
            schema_ok = check_and_fix_delta_schema(delta_path, table_name)

            if not schema_ok:
                # Schema incorreto, recriar tabela
                print("Recriando tabela Delta com schema correto...")
                df_processed = prepare_dataframe_for_merge(df, table_name)

                from pyspark.sql.functions import year, month, current_timestamp

                df_with_partition = df_processed.withColumn("DT_INGESTION", current_timestamp()) \
                                              .withColumn("INGESTION_YEAR", year(col("DT_INGESTION"))) \
                                              .withColumn("INGESTION_MONTH", month(col("DT_INGESTION")))

                print(f"Particionamento criado com DT_INGESTION")

                df_with_partition.write.format("delta") \
                       .mode("overwrite") \
                       .option("overwriteSchema", "true") \
                       .partitionBy("INGESTION_YEAR", "INGESTION_MONTH") \
                       .save(delta_path)
                print("Tabela Delta recriada com sucesso")
            else:
                # Schema correto, fazer merge incremental
                print("Schema correto, preparando para merge incremental...")
                target_schema = get_delta_schema_for_merge(delta_path)

                # Preparar DataFrame para merge
                merge_df = prepare_dataframe_for_merge(df, table_name, target_schema)
                if not merge_df:
                    print("Nenhum DataFrame compatível para merge")
                    return None

                # Executar merge com colunas específicas
                print("Executando merge Delta...")
                delta_table = DeltaTable.forPath(spark, delta_path)
                merge_success = execute_merge_with_specific_columns(delta_table, merge_df, target_schema)
                if not merge_success:
                    print("Falha no merge Delta")
                    return None

        # Criar ou atualizar tabela Unity Catalog (PRESERVANDO MODIFICAÇÕES)
        create_or_update_unity_catalog(full_table_name, delta_path)

        return df
    except Exception as e:
        print(f"Erro no LOAD para bronze {table_name}: {e}")
        return None

def process_el_unity_incremental_gov(table_name):
    # Executa o pipeline EL Unity + Incremental + GOV
    print(f"Iniciando process_el_unity_incremental_gov para {table_name}")
    
    stage_df = extract_from_staging(table_name)
    if not stage_df:
        print(f"Falha no EXTRACT de staging para {table_name}")
        return None
    
    print(f"EXTRACT concluído com sucesso para {table_name}")
    result_df = load_to_bronze_unity_incremental_gov(stage_df, table_name)
    
    if result_df:
        print(f"Pipeline EL concluído com sucesso para {table_name}")
    else:
        print(f"Falha no pipeline EL para {table_name}")
    
    return result_df

def query_bronze_unity(table_name):
    # Consulta na tabela bronze Unity Catalog
    try:
        full_table_name = f"{CATALOG_NAME}.{SCHEMA_NAME}.{table_name}"
        count_query = f"SELECT COUNT(*) as total FROM {full_table_name}"
        count_result = spark.sql(count_query)
        count_result.show()
        
    except Exception as e:
        print(f"Erro ao consultar tabela bronze: {e}")

def show_delta_info(table_name):
    # Mostra informações da tabela Delta
    try:
        delta_path = f"s3://{S3_BRONZE_PATH}/{table_name}"
        from delta.tables import DeltaTable
        delta_table = DeltaTable.forPath(spark, delta_path)
        history = delta_table.history()
        print(f"Versões Delta: {history.count()}")
            
    except Exception as e:
        print(f"Erro ao mostrar informações Delta: {e}")

def show_sample_data(table_name):
    # Mostra dados de exemplo da tabela
    try:
        full_table_name = f"{CATALOG_NAME}.{SCHEMA_NAME}.{table_name}"
        sample_query = f"SELECT * FROM {full_table_name} LIMIT 5"
        print("Dados de exemplo:")
        spark.sql(sample_query).show(truncate=False)
        
    except Exception as e:
        print(f"Erro ao mostrar dados de exemplo: {e}")

def show_governance_info():
    # Mostra informações sobre a governança aplicada
    print("\n=== INFORMAÇÕES DE GOVERNAÇA GOV ===")
    print(f"Total de colunas mapeadas: {len(COLUMN_MAPPING)}")
    print("Padrões aplicados:")
    print("- NME_: Nomes e descrições")
    print("- COD_: Códigos e identificadores")
    print("- ID_: Identificadores únicos")
    print("- FLG_: Flags booleanos")
    print("- DT_: Datas e timestamps")
    print("- DESC_: Descrições e textos")
    
    # Mostrar mapeamento específico das colunas principais
    print("\n=== MAPEAMENTO PRINCIPAL DE COLUNAS ===")
    print("Identificação:")
    print(f"  subtype_name → NME_SUBTYPE")
    
    print("\nMetadados:")
    print(f"  source → NME_SOURCE")
    print(f"  ingestion_timestamp → DT_INGESTION")
    
    print("\nDatas:")
    print(f"  created_at → DT_CREATED")
    print(f"  updated_at → DT_UPDATED")
    
    print("\n=== FILTRO TEMPORAL ===")
    print("Dados de referência MTG API - sem filtro temporal")
    print("Subtypes são dados estáticos de referência")
    
    print("\n=== PARTICIONAMENTO ===")
    print("Particionamento por INGESTION_YEAR e INGESTION_MONTH")
    print("Baseado em DT_INGESTION para dados de referência")

def check_and_fix_delta_schema(delta_path, table_name):
    # Verifica se o schema da tabela Delta está correto, se não estiver, deleta e recria
    try:
        df = spark.read.format("delta").load(delta_path)
        schema_fields = [field.name for field in df.schema.fields]
        print(f"Schema atual da tabela Delta: {schema_fields}")

        # Definir schema esperado para subtypes
        if table_name == "TB_BRONZE_SUBTYPES":
            expected_schema = ['NME_SUBTYPE', 'DT_INGESTION', 'NME_SOURCE', 'INGESTION_YEAR', 'INGESTION_MONTH']
        else:
            return True  # Para outros, não faz nada

        # Verificar se todas as colunas esperadas estão presentes e se não há colunas antigas
        missing_columns = [col for col in expected_schema if col not in schema_fields]
        incorrect_columns = [col for col in schema_fields if col in ['endpoint', 'source', 'ingestion_timestamp']]

        if missing_columns or incorrect_columns:
            print(f"Schema incorreto detectado:")
            if missing_columns:
                print(f"  - Colunas faltando: {missing_columns}")
            if incorrect_columns:
                print(f"  - Colunas com nomenclatura incorreta: {incorrect_columns}")
            print("Deletando tabela Delta para recriação...")
            dbutils.fs.rm(delta_path, recurse=True)
            print(f"Tabela Delta deletada: {delta_path}")
            return False  # Precisa recriar
        else:
            print("✅ Schema correto - Tabela não precisa ser recriada")
            return True

    except Exception as e:
        print(f"Erro ao verificar schema Delta: {e}")
        return False

# =============================================================================
# EXECUÇÃO PRINCIPAL
# =============================================================================

try:
    spark
except NameError:
    from pyspark.sql import SparkSession
    spark = SparkSession.builder.getOrCreate()

print("Iniciando pipeline EL Unity com nomenclatura padronizada - TB_BRONZE_SUBTYPES")
print("Dados de referência MTG API - sem filtro temporal")

setup_success = setup_unity_catalog()
if not setup_success:
    raise Exception("Falha ao configurar Unity Catalog")

# Executar pipeline
subtypes_bronze_df = process_el_unity_incremental_gov("TB_BRONZE_SUBTYPES")

if subtypes_bronze_df:
    print("Pipeline executado com sucesso")
    query_bronze_unity("TB_BRONZE_SUBTYPES")
    show_delta_info("TB_BRONZE_SUBTYPES")
    show_sample_data("TB_BRONZE_SUBTYPES")
    show_governance_info()
else:
    print("Falha no pipeline") 

In [0]:
# =============================================================================
# FUNÇÕES UTILITÁRIAS
# =============================================================================
def setup_unity_catalog():
    # Configura o Unity Catalog e schema
    try:
        spark.sql(f"CREATE CATALOG IF NOT EXISTS {CATALOG_NAME}")
        spark.sql(f"USE CATALOG {CATALOG_NAME}")
        spark.sql(f"CREATE SCHEMA IF NOT EXISTS {SCHEMA_NAME}")
        spark.sql(f"USE SCHEMA {SCHEMA_NAME}")
        return True
    except Exception as e:
        print(f"Erro ao configurar Unity Catalog: {e}")
        return False

def extract_from_staging(table_name):
    # EXTRACT: Lê dados da camada staging
    try:
        stage_path = f"{S3_STAGE_PATH}/*_{table_name}.parquet"
        df = spark.read.parquet(stage_path)
        print(f"Extraídos {df.count()} registros de staging para {table_name}")
        return df
    except Exception as e:
        print(f"Erro no EXTRACT de staging: {e}")
        return None

def apply_temporal_filter(df, table_name):
    # Aplica filtro temporal de 5 anos
    if table_name == "subtypes":
        # Para subtypes, não aplicar filtro temporal por enquanto (subtypes não têm releaseDate)
        # Os subtypes são dados de referência que não mudam com o tempo
        print(f"Filtro temporal não aplicado para subtypes (dados de referência)")
        return df
    return df

def check_delta_table_exists(delta_path):
    # Verifica se a tabela Delta existe
    try:
        # Verificar se _delta_log existe
        delta_log_path = f"{delta_path}/_delta_log"
        try:
            files = dbutils.fs.ls(delta_log_path)
            if files:
                # Tentar ler a tabela para verificar se tem dados
                df = spark.read.format("delta").load(delta_path)
                count = df.count()
                print(f"Tabela Delta existe com {count} registros")
                return True, count > 0
            else:
                return False, False
        except:
            return False, False
    except Exception as e:
        print(f"Erro ao verificar tabela Delta: {e}")
        return False, False

def get_delta_schema_for_merge(delta_path):
    # Obtém o schema da tabela Delta existente, excluindo colunas de particionamento
    try:
        df = spark.read.format("delta").load(delta_path)
        schema_fields = [field.name for field in df.schema.fields]
        
        # Excluir colunas de particionamento do schema para merge
        partition_columns = ['partition_year', 'partition_month']
        merge_schema = [col for col in schema_fields if col not in partition_columns]
        
        print(f"Schema completo da tabela Delta: {schema_fields}")
        print(f"Schema para merge (sem particionamento): {merge_schema}")
        return merge_schema
    except Exception as e:
        print(f"Erro ao obter schema Delta: {e}")
        return []

def prepare_dataframe_for_merge(df, table_name, target_schema=None):
    # Prepara DataFrame para merge, garantindo compatibilidade de schema
    if not df:
        return None
    
    try:
        # Determinar a coluna de chave baseada na tabela
        if table_name == "subtypes":
            key_column = 'subtype_name'
        else:
            key_column = 'name'
        
        # Elimina duplicidade baseada na chave correta
        total_before = df.count()
        df = df.dropDuplicates([key_column])
        total_after = df.count()
        print(f"Removidas {total_before - total_after} duplicatas baseadas em '{key_column}'")
        
        # Aplicar filtro temporal
        df = apply_temporal_filter(df, table_name)
        
        # Se temos schema de destino, filtrar colunas compatíveis
        if target_schema:
            available_columns = df.columns
            compatible_columns = [col for col in available_columns if col in target_schema]
            
            if not compatible_columns:
                print("Nenhuma coluna compatível encontrada para merge")
                return None
            
            print(f"Colunas compatíveis para merge: {compatible_columns}")
            df = df.select(compatible_columns)
        
        return df
    except Exception as e:
        print(f"Erro ao preparar DataFrame para merge: {e}")
        return None

def execute_merge_with_specific_columns(delta_table, merge_df, merge_columns, table_name):
    # Executa merge especificando exatamente quais colunas atualizar
    try:
        # Determinar a chave de merge baseada na tabela
        if table_name == "subtypes":
            # Verificar se a tabela existente usa 'name' ou 'subtype_name'
            existing_schema = [field.name for field in delta_table.toDF().schema.fields]
            print(f"Schema da tabela existente: {existing_schema}")
            
            if 'name' in existing_schema and 'subtype_name' not in existing_schema:
                # Tabela antiga usa 'name', mas novos dados usam 'subtype_name'
                # Renomear a coluna no DataFrame de origem para compatibilidade
                print("Renomeando 'subtype_name' para 'name' para compatibilidade com tabela existente")
                merge_df = merge_df.withColumnRenamed('subtype_name', 'name')
                merge_condition = "bronze.name = novo.name"
                key_column = 'name'
                
                # Atualizar merge_columns para refletir a renomeação
                merge_columns = [col if col != 'subtype_name' else 'name' for col in merge_columns]
            else:
                # Tabela usa 'subtype_name'
                merge_condition = "bronze.subtype_name = novo.subtype_name"
                key_column = 'subtype_name'
        else:
            # Para outras tabelas, usar a lógica padrão
            merge_condition = "bronze.name = novo.name"
            key_column = 'name'
        
        # Filtrar colunas compatíveis após a renomeação
        if merge_columns:
            available_columns = merge_df.columns
            compatible_columns = [col for col in available_columns if col in merge_columns]
            
            if not compatible_columns:
                print("Nenhuma coluna compatível encontrada para merge")
                return False
            
            print(f"Colunas compatíveis para merge: {compatible_columns}")
            merge_df = merge_df.select(compatible_columns)
        
        # Construir as ações de UPDATE especificando colunas
        update_actions = {}
        for col_name in merge_df.columns:
            if col_name != key_column:  # Não atualizar a chave de merge
                update_actions[col_name] = f"novo.{col_name}"
        
        # Construir as ações de INSERT especificando colunas
        insert_actions = {}
        for col_name in merge_df.columns:
            insert_actions[col_name] = f"novo.{col_name}"
        
        print(f"Executando merge com {len(update_actions)} colunas para UPDATE")
        print(f"Colunas para UPDATE: {list(update_actions.keys())}")
        print(f"Condição de merge: {merge_condition}")
        print(f"Colunas do DataFrame de origem após preparação: {merge_df.columns}")
        
        # Executar merge com ações específicas
        delta_table.alias("bronze").merge(
            merge_df.alias("novo"),
            merge_condition
        ).whenMatchedUpdate(set=update_actions) \
         .whenNotMatchedInsert(values=insert_actions) \
         .execute()
        
        print("Merge Delta executado com sucesso")
        return True
    except Exception as e:
        print(f"Erro no merge Delta: {e}")
        return False

def check_unity_table_exists(full_table_name):
    # Verifica se a tabela Unity Catalog existe
    try:
        # Listar tabelas no schema
        tables_df = spark.sql(f"SHOW TABLES IN {CATALOG_NAME}.{SCHEMA_NAME}")
        tables_list = [row.tableName for row in tables_df.collect()]
        exists = TABLE_NAME in tables_list
        print(f"Tabela Unity Catalog '{full_table_name}' existe: {exists}")
        return exists
    except Exception as e:
        print(f"Erro ao verificar tabela Unity Catalog: {e}")
        return False

def create_or_update_unity_catalog(full_table_name, delta_path):
    # Cria ou atualiza tabela Unity Catalog preservando modificações existentes
    try:
        table_exists = check_unity_table_exists(full_table_name)
        
        if not table_exists:
            # Criar tabela pela primeira vez
            print(f"Criando tabela Unity Catalog: {full_table_name}")
            create_table_sql = f"""
            CREATE TABLE {full_table_name}
            USING DELTA
            LOCATION '{delta_path}'
            COMMENT 'Tabela bronze de subtypes do Magic: The Gathering'
            """
            spark.sql(create_table_sql)
            print(f"Tabela Unity Catalog criada: {full_table_name}")
        else:
            # Tabela já existe, apenas atualizar propriedades de pipeline (sem perder modificações)
            print(f"Tabela Unity Catalog já existe: {full_table_name}")
            print("Atualizando apenas propriedades de pipeline (preservando modificações existentes)")
        
        # Sempre atualizar propriedades de pipeline (não afeta modificações customizadas)
        try:
            spark.sql(f"""
            ALTER TABLE {full_table_name} SET TBLPROPERTIES (
                'bronze_layer' = 'true',
                'data_source' = 'mtg_api',
                'last_processing_date' = '{datetime.now().strftime("%Y-%m-%d %H:%M:%S")}',
                'table_type' = 'bronze',
                'load_mode' = 'incremental_merge_fixed',
                'temporal_window_years' = '{YEARS_BACK}',
                'partitioning' = 'ingestion_timestamp_year_month'
            )
            """)
            print("Propriedades de pipeline atualizadas")
        except Exception as e:
            print(f"Aviso: Não foi possível atualizar propriedades de pipeline: {e}")
            
    except Exception as e:
        print(f"Erro ao criar/atualizar tabela Unity Catalog: {e}")

def load_to_bronze_unity_incremental_fixed(df, table_name):
    # LOAD: Carrega dados na camada bronze (Unity + Incremental Corrigido)
    if not df:
        return None
    try:
        delta_path = f"s3://{S3_BRONZE_PATH}/{table_name}"
        full_table_name = f"{CATALOG_NAME}.{SCHEMA_NAME}.{table_name}"
        from delta.tables import DeltaTable
        
        # Verificar se tabela Delta existe
        delta_exists, has_data = check_delta_table_exists(delta_path)
        
        if not delta_exists or not has_data:
            # Primeira criação com particionamento
            print("Criando tabela Delta pela primeira vez...")
            df_processed = prepare_dataframe_for_merge(df, table_name)
            
            if table_name == "subtypes":
                # Para subtypes, usar ingestion_timestamp para particionamento (dados de referência)
                df_with_partition = df_processed.withColumn("partition_year", 
                                  year(col("ingestion_timestamp"))) \
                       .withColumn("partition_month", 
                                  month(col("ingestion_timestamp")))
                
                df_with_partition.write.format("delta") \
                       .mode("overwrite") \
                       .option("overwriteSchema", "true") \
                       .partitionBy("partition_year", "partition_month") \
                       .save(delta_path)
            else:
                df_processed.write.format("delta") \
                       .mode("overwrite") \
                       .option("overwriteSchema", "true") \
                       .save(delta_path)
            print("Tabela Delta criada com sucesso")
        else:
            # Para merge, obter schema da tabela existente (excluindo particionamento) e preparar dados
            print("Tabela Delta existe, preparando para merge...")
            target_schema = get_delta_schema_for_merge(delta_path)
            
            # Preparar DataFrame para merge (sem filtrar colunas ainda)
            merge_df = prepare_dataframe_for_merge(df, table_name, None)  # Não filtrar colunas ainda
            if not merge_df:
                print("Nenhum DataFrame compatível para merge")
                return None
            
            # Executar merge com colunas específicas (sem particionamento)
            print("Executando merge Delta...")
            delta_table = DeltaTable.forPath(spark, delta_path)
            
            # Usar função específica para merge com colunas definidas
            merge_success = execute_merge_with_specific_columns(delta_table, merge_df, target_schema, table_name)
            if not merge_success:
                print("Falha no merge Delta")
                return None

        # Criar ou atualizar tabela Unity Catalog (PRESERVANDO MODIFICAÇÕES)
        create_or_update_unity_catalog(full_table_name, delta_path)
        
        return df
    except Exception as e:
        print(f"Erro no LOAD para bronze {table_name}: {e}")
        return None

def process_el_unity_incremental_fixed(table_name):
    # Executa o pipeline EL Unity + Incremental Corrigido
    stage_df = extract_from_staging(table_name)
    if not stage_df:
        print(f"Falha no EXTRACT de staging para {table_name}")
        return None
    
    result_df = load_to_bronze_unity_incremental_fixed(stage_df, table_name)
    return result_df

def query_bronze_unity(table_name):
    # Consulta na tabela bronze Unity Catalog
    try:
        full_table_name = f"{CATALOG_NAME}.{SCHEMA_NAME}.{table_name}"
        count_query = f"SELECT COUNT(*) as total FROM {full_table_name}"
        count_result = spark.sql(count_query)
        count_result.show()
        
        # Mostrar informações de particionamento para subtypes
        if table_name == "subtypes":
            partition_query = f"""
            SELECT 
                partition_year,
                partition_month,
                COUNT(*) as records
            FROM {full_table_name}
            GROUP BY partition_year, partition_month
            ORDER BY partition_year DESC, partition_month DESC, records DESC
            LIMIT 10
            """
            print("Top 10 partições por volume de dados:")
            spark.sql(partition_query).show()
        
    except Exception as e:
        print(f"Erro ao consultar tabela bronze: {e}")

def show_delta_info(table_name):
    # Mostra informações da tabela Delta
    try:
        delta_path = f"s3://{S3_BRONZE_PATH}/{table_name}"
        from delta.tables import DeltaTable
        delta_table = DeltaTable.forPath(spark, delta_path)
        history = delta_table.history()
        print(f"Versões Delta: {history.count()}")
            
    except Exception as e:
        print(f"Erro ao mostrar informações Delta: {e}")

def show_sample_data(table_name):
    # Mostra dados de exemplo da tabela
    try:
        full_table_name = f"{CATALOG_NAME}.{SCHEMA_NAME}.{table_name}"
        sample_query = f"SELECT * FROM {full_table_name} LIMIT 5"
        print("Dados de exemplo:")
        spark.sql(sample_query).show(truncate=False)
        
    except Exception as e:
        print(f"Erro ao mostrar dados de exemplo: {e}")



In [0]:
# =============================================================================
# EXECUÇÃO PRINCIPAL
# =============================================================================

try:
    spark
except NameError:
    from pyspark.sql import SparkSession
    spark = SparkSession.builder.getOrCreate()

setup_success = setup_unity_catalog()
if not setup_success:
    raise Exception("Falha ao configurar Unity Catalog")

subtypes_bronze_df = process_el_unity_incremental_fixed("subtypes")

if subtypes_bronze_df:
    print("Pipeline executado com sucesso")
    query_bronze_unity("subtypes")
    show_delta_info("subtypes")
    show_sample_data("subtypes")
else:
    print("Falha no pipeline") 